### Collaborative filtering


In [59]:
import pandas as pd
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [60]:
#path to the mindsmall training data
training_dataset = '../MINDsmall_train'

validation_dataset = '../MINDsmall_train'

entity_embedding = pd.read_csv("../MINDsmall_train/entity_embedding.vec", sep='\t')
relation_embedding = pd.read_csv("../MINDsmall_train/relation_embedding.vec", sep='\t')


news_data = pd.read_csv("../MINDsmall_train/news.tsv",
    sep='\t',
    names=["newsId", "category", "subcategory", "title","abstract", "url", "title_entities","abstract_entities"]
)

behaviors_data = pd.read_csv(
    "../MINDsmall_train/behaviors.tsv",
    sep='\t',
    names=["impressionId", "userId", "timestamp", "click_history", "impressions"],
    parse_dates=['timestamp'] 
)

### Preprocess data

In [61]:
behaviors_data['timestamp'] = pd.to_datetime(behaviors_data['timestamp'], format='%Y-%m-%d %H:%M:%S')
behaviors_data['clicks'] = behaviors_data['click_history'].str.split().str.len()
behaviors_data['impressions_list'] = behaviors_data['impressions'].str.split()
behaviors_data['impressions_count'] = behaviors_data['impressions_list'].str.len()

# remove null values
news_data = news_data.dropna().reset_index(drop=True)


### Remove the null values for the clicks & impressions

In [62]:
null_clicks_rows = behaviors_data[behaviors_data['clicks'].isnull() & behaviors_data['click_history'].isnull()]

# Remove the rows with null values
behaviors_data = behaviors_data.dropna().reset_index(drop=True)

In [63]:
# sort behaviors_data by timestamp
behaviors_data = behaviors_data.sort_values(by='timestamp')

In [64]:
news_data.head()

,newsId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [65]:
behaviors_data.head()

,impressionId,userId,timestamp,click_history,impressions,clicks,impressions_list,impressions_count
19705,20112,U65916,2019-11-09 00:00:19,N51706 N40767 N12096 N9798 N38802 N54827 N5780...,N54300-0 N46057-1 N57005-0 N52154-0 N57099-0 N...,15.0,"[N54300-0, N46057-1, N57005-0, N52154-0, N5709...",31
13531,13807,U49985,2019-11-09 00:01:13,N5056 N29975 N53234 N39603 N50032 N8422 N53580...,N20602-0 N50059-0 N57768-1 N50135-1 N15134-0 N...,93.0,"[N20602-0, N50059-0, N57768-1, N50135-1, N1513...",29
27115,27660,U25550,2019-11-09 00:02:44,N17260 N38298 N33976 N47719 N14888 N18870 N4607,N50135-0 N15134-0 N52433-1 N20602-0 N64536-0,7.0,"[N50135-0, N15134-0, N52433-1, N20602-0, N6453...",5
149080,152217,U19710,2019-11-09 00:02:50,N3530 N48284 N43019 N62546 N138 N13138 N10676 ...,N57099-0 N30295-0 N21086-0 N5379-0 N57005-0 N4...,8.0,"[N57099-0, N30295-0, N21086-0, N5379-0, N57005...",56
41348,42166,U38106,2019-11-09 00:03:09,N16874 N264 N48697 N51366,N3491-0 N20602-0 N25785-0 N23575-0 N38783-0 N1...,4.0,"[N3491-0, N20602-0, N25785-0, N23575-0, N38783...",48


In [71]:
clicked_articles = []


for index, row in behaviors_data.iterrows():
    user_id = row['userId']
    impressions = row['impressions_list']
    # add the articles that has clicks
    for impression in impressions:
        article_id, clicked = impression.split('-')
        if clicked == '1': 
            clicked_articles.append((user_id, article_id))

# dataframe of clicked articles
clicks_df = pd.DataFrame(clicked_articles, columns=['UserID', 'ArticleID'])
# column for clicks
clicks_df['Click'] = 1  

# matrix of user-item interactions
user_item_matrix = clicks_df.pivot_table(index='ArticleID', columns='UserID', values='Click', fill_value=0)

# convert the user-item matrix to sparse matrix
user_item_sparse = csr_matrix(user_item_matrix.values)

# view the user-item matrix 
user_item_matrix.head(20)


UserID,U100,U1000,U10001,U10003,U10008,U10010,U10011,U10012,U10014,U10015,...,U9987,U9988,U999,U9991,U9992,U9993,U9995,U9996,U9997,U9999
ArticleID,,,,,,,,,,,,,,,,,,,,,
N10032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N10051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N10056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N10057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N1006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N10061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N10067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N10070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N10072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
# view the user_item_sparse matrix
user_item_sparse

<7659x49108 sparse matrix of type '<class 'numpy.float64'>'
	with 229700 stored elements in Compressed Sparse Row format>

In [68]:
# Assuming 'user_item_matrix' is your user-item interaction matrix
# Compute the cosine similarity matrix across items
item_similarity_matrix = cosine_similarity(user_item_sparse.T)

# Convert the similarity matrix to a DataFrame for easier manipulation
item_similarity_df = pd.DataFrame(item_similarity_matrix, index=user_item_matrix.columns, columns=user_item_matrix.columns)

In [69]:
def recommend_items(user_id, user_item_matrix, item_similarity_df, N=5):
    # Get the items that the user has interacted with
    interacted_items = user_item_matrix.loc[:, user_id].copy()
    interacted_items = interacted_items[interacted_items > 0].index.tolist()
    
    # Ensure the items are in the item similarity matrix
    interacted_items = [item for item in interacted_items if item in item_similarity_df.columns]
    
    # Calculate a score for each item based on similarity
    if interacted_items:  # Ensure there's at least one interacted item in the similarity matrix
        item_scores = item_similarity_df[interacted_items].sum(axis=1).sort_values(ascending=False)
        # Remove items the user has already interacted with
        item_scores = item_scores.drop(interacted_items, errors='ignore')
    else:
        item_scores = pd.Series()  # If no interacted items are found, return an empty series
    
    # Get the top N items with the highest score
    top_n_items = item_scores.head(N).index.tolist()
    
    return top_n_items

In [70]:
recommend_items('U89744', user_item_matrix, item_similarity_df, N=50)

[]